<a href="https://colab.research.google.com/github/Frynnn-69/mini-research-bdc2025/blob/main/src/03_modeling/03_audio_model_baseline(v1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [1]:
# =================================================================
# Warming up + Setup
# =================================================================
from google.colab import drive
import os

drive.mount('/content/drive')

repo_path = '/content/drive/My Drive/mini-research-bdc2025/'
os.chdir(repo_path)

!git reset --hard origin/main
!git pull

notebook_folder = 'src/03_modeling/'

working_dir = os.path.join(repo_path, notebook_folder)

os.makedirs(working_dir, exist_ok=True)
os.chdir(working_dir)

!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Updating files: 100% (17/17), done.
HEAD is now at e24d736 .
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 4.70 KiB | 15.00 KiB/s, done.
From https://github.com/frynnn-69/mini-research-bdc2025
   e24d736..53a032a  main       -> origin/main
Updating e24d736..53a032a
Fast-forward
 src/datatest.csv | 201 +++++++++++++++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 201 insertions(+)
 create mode 100644 src/datatest.csv
/content/drive/My Drive/mini-research-bdc2025/src/03_modeling


In [2]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

audio_train_path = '/content/drive/My Drive/BDC_DATASET/audio/train'
csv_path = '/content/drive/My Drive/mini-research-bdc2025/src/02_data_processing/datatrain_final.csv'

df_labels = pd.read_csv(csv_path)
print(f"Number of labels loaded: {len(df_labels)}")

try:
    audio_files = os.listdir(audio_train_path)
    print(f"Number of audio files found: {len(audio_files)}")

    audio_ids = [int(f.replace('audio_', '').replace('.wav', '')) for f in audio_files]
    df_audio = pd.DataFrame({
        'id': audio_ids,
        'audio_path': [os.path.join(audio_train_path, f) for f in audio_files]
    })

    df_model_data = pd.merge(df_labels, df_audio, on='id')
    print(f"data ready to train (audio & labels): {len(df_model_data)}")

    display(df_model_data.head())

except FileNotFoundError:
    print(f"ERROR")

Number of labels loaded: 755
Number of audio files found: 752
data ready to train (audio & labels): 752


,id,video,emotion,audio_path
0,1,https://www.instagram.com/reel/DNKcHgdA-d1/?ig...,Surprise,/content/drive/My Drive/BDC_DATASET/audio/trai...
1,2,https://www.instagram.com/reel/DNHwrh2gnBm/?ig...,Surprise,/content/drive/My Drive/BDC_DATASET/audio/trai...
2,3,https://www.instagram.com/reel/DM7QsjnRCoa/?ig...,Surprise,/content/drive/My Drive/BDC_DATASET/audio/trai...
3,4,https://www.instagram.com/reel/DNBBEt6Paxj/?ig...,Surprise,/content/drive/My Drive/BDC_DATASET/audio/trai...
4,5,https://www.instagram.com/reel/DMz13fQzZsN/?ig...,Proud,/content/drive/My Drive/BDC_DATASET/audio/trai...


In [3]:
if 'df_model_data' in locals():
    train_df, val_df = train_test_split(
        df_model_data,
        test_size=0.2,
        random_state=42,
        stratify=df_model_data['emotion']
    )

    print(f"Train: {len(train_df)}")
    print(f"Validation: {len(val_df)}")

    print("\nProportion of emotions in training data:")
    print(train_df['emotion'].value_counts(normalize=True))
    print("\nProportion of emotions in validation data:")
    print(val_df['emotion'].value_counts(normalize=True))
else:
    print("ERROR: Dataframe 'df_model_data' not found.")

Train: 601
Validation: 151

Proportion of emotions in training data:
emotion
Surprise    0.417637
Trust       0.217970
Proud       0.198003
Joy         0.068220
Anger       0.048253
Sadness     0.021631
Fear        0.019967
Neutral     0.008319
Name: proportion, dtype: float64

Proportion of emotions in validation data:
emotion
Surprise    0.423841
Trust       0.218543
Proud       0.198675
Joy         0.066225
Anger       0.046358
Sadness     0.019868
Fear        0.019868
Neutral     0.006623
Name: proportion, dtype: float64


In [4]:
!pip install -q datasets torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.4 MB/s eta 0:00:00


In [5]:
from datasets import Dataset, Audio

if 'train_df' in locals() and 'val_df' in locals():
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)

    print("Training Dataset:")
    print(train_dataset)
    print("\nValidation Dataset:")
    print(val_dataset)

    train_dataset = train_dataset.cast_column("audio_path", Audio(sampling_rate=16000))
    val_dataset = val_dataset.cast_column("audio_path", Audio(sampling_rate=16000))

    print(train_dataset[0])

    labels = df_model_data['emotion'].unique()
    label2id = {label: i for i, label in enumerate(labels)}
    id2label = {i: label for i, label in enumerate(labels)}

    print("label2id:", label2id)
    print("id2label:", id2label)

else:
    print("ERROR: DataFrame 'train_df' or 'val_df' not found.")

Training Dataset:
Dataset({
    features: ['id', 'video', 'emotion', 'audio_path', '__index_level_0__'],
    num_rows: 601
})

Validation Dataset:
Dataset({
    features: ['id', 'video', 'emotion', 'audio_path', '__index_level_0__'],
    num_rows: 151
})
{'id': 692, 'video': 'https://www.instagram.com/reel/DJofVAHTuMI/?igsh=Y2hvdHd0cHluMXFs', 'emotion': 'Trust', 'audio_path': <datasets.features._torchcodec.AudioDecoder object at 0x7e2a615e1e80>, '__index_level_0__': 658}
label2id: {'Surprise': 0, 'Proud': 1, 'Trust': 2, 'Sadness': 3, 'Joy': 4, 'Anger': 5, 'Fear': 6, 'Neutral': 7}
id2label: {0: 'Surprise', 1: 'Proud', 2: 'Trust', 3: 'Sadness', 4: 'Joy', 5: 'Anger', 6: 'Fear', 7: 'Neutral'}


#Modeling & Training

In [6]:
!pip install -q transformers accelerate

In [7]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification

model_checkpoint = "facebook/wav2vec2-base"
num_labels = len(label2id)

if 'label2id' in locals():
    # 1. Unduh "pisau dapur" (Feature Extractor)
    feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
    print(feature_extractor)

    # 2. Unduh "koki" (Model)
    model = AutoModelForAudioClassification.from_pretrained(
        model_checkpoint,
        num_labels=num_labels,
        label2id=label2id,
        id2label=id2label,
    )

    #print(model)

else:
    print("ERROR: Variabel 'label2id' not found.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}



pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
if 'feature_extractor' in locals():
    def preprocess_function(examples):
        audio_arrays = [x["array"] for x in examples["audio_path"]]
        inputs = feature_extractor(
            audio_arrays,
            sampling_rate=feature_extractor.sampling_rate,
            max_length=16000 * 5,
            truncation=True,
            padding=True
        )

        inputs["labels"] = [label2id[label] for label in examples["emotion"]]
        return inputs

    encoded_train_dataset = train_dataset.map(preprocess_function, remove_columns=["video", "emotion", "audio_path", "__index_level_0__"], batched=True)

    encoded_val_dataset = val_dataset.map(preprocess_function, remove_columns=["video", "emotion", "audio_path", "__index_level_0__"], batched=True)

    print(encoded_train_dataset[0])

else:
    print("ERROR: Variabel 'feature_extractor' not found.")

Map:   0%|          | 0/601 [00:00<?, ? examples/s]

Map:   0%|          | 0/151 [00:00<?, ? examples/s]

{'id': 692, 'input_values': [0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.0005780447390861809, 0.000

In [9]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00


In [10]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis=1)

    acc_score = accuracy.compute(predictions=predictions, references=labels)

    f1_score = f1.compute(predictions=predictions, references=labels, average="macro")

    return {
        "accuracy": acc_score["accuracy"],
        "f1_macro": f1_score["f1"],
    }

In [11]:
from transformers import TrainingArguments, Trainer

repository_id = "wav2vec2-base-bdc2025-finetuned-audio-emotion"

training_args = TrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    learning_rate=3e-5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    push_to_hub=False,

    eval_strategy="steps",
    eval_steps=76,
    save_strategy="steps",
    save_steps=76,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_val_dataset,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-1986575526.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()
#trainer.train(resume_from_checkpoint=True)

Step,Training Loss,Validation Loss,Accuracy,F1 Macro
76,1.578400,1.602867,0.423841,0.074419
152,1.618300,1.547175,0.423841,0.074419
228,1.599400,1.533213,0.423841,0.074419
304,1.633400,1.531767,0.423841,0.074419
380,1.438800,1.534517,0.423841,0.074419
456,1.658100,1.535318,0.423841,0.074419
532,1.723700,1.537827,0.423841,0.074419
608,1.577200,1.523124,0.423841,0.074766
684,1.695400,1.526351,0.423841,0.088033
760,1.427400,1.527862,0.423841,0.093048


TrainOutput(global_step=760, training_loss=1.5731440230419762, metrics={'train_runtime': 1638.9484, 'train_samples_per_second': 3.667, 'train_steps_per_second': 0.464, 'total_flos': 2.728173591936e+17, 'train_loss': 1.5731440230419762, 'epoch': 10.0})

#Testing


In [13]:
test_csv_path = '/content/drive/MyDrive/mini-research-bdc2025/src/datatest.csv'
base_storage_path = '/content/drive/My Drive/BDC_DATASET/'
video_test_path = os.path.join(base_storage_path, 'videos', 'test')
audio_test_path = os.path.join(base_storage_path, 'audio', 'test')

os.makedirs(video_test_path, exist_ok=True)
os.makedirs(audio_test_path, exist_ok=True)

df_test = pd.read_csv(test_csv_path)
print(f"\n{len(df_test)} test video found.")


200 test video found.


In [14]:
!pip install -q yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.3 MB/s eta 0:00:00


In [15]:

import subprocess
import time
import random

if 'df_test' in locals():
    for index, row in df_test.iterrows():
        video_id = row['id']
        video_url = row['video']

        output_audio_path = os.path.join(audio_test_path, f"audio_{video_id}.wav")

        if os.path.exists(output_audio_path):
            print(f"Audio ID {video_id}: .")
            continue

        video_output_template = os.path.join(video_test_path, f"video_{video_id}.%(ext)s")
        cookies_file_path = '/cookies.txt'

        command_dl = f"yt-dlp -q --cookies '{cookies_file_path}' -f 'bv*[ext=mp4]+ba[ext=m4a]/b[ext=mp4] / bv*+ba/b' -o '{video_output_template}' '{video_url}'"

        try:
            # Unduh Vid
            subprocess.run(command_dl, shell=True, check=True, capture_output=True, text=True)

            # Ekstrak Aud
            downloaded_video_file = None
            for fname in os.listdir(video_test_path):
                if fname.startswith(f"video_{video_id}."):
                    downloaded_video_file = os.path.join(video_test_path, fname)
                    break

            if downloaded_video_file:
                command_ffmpeg = f"ffmpeg -i '{downloaded_video_file}' -vn -acodec pcm_s16le -ar 16000 -ac 1 '{output_audio_path}'"
                subprocess.run(command_ffmpeg, shell=True, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                print(f"Video ID {video_id}: Successfully ")
                os.remove(downloaded_video_file)
            else:
                print(f"Video ID {video_id}: Failed")

        except subprocess.CalledProcessError as e:
            error_lines = e.stderr.strip().split('\n')
            print(f"Video ID {video_id}:{error_lines[-1]}")

        time.sleep(random.uniform(1, 3))
else:
    print("Error")

Memulai pipeline untuk 200 video tes dengan mode 'sopan'...
Audio ID 1: Sudah ada, dilewati.
Audio ID 2: Sudah ada, dilewati.
Audio ID 3: Sudah ada, dilewati.
Audio ID 4: Sudah ada, dilewati.
Audio ID 5: Sudah ada, dilewati.
Audio ID 6: Sudah ada, dilewati.
Audio ID 7: Sudah ada, dilewati.
Audio ID 8: Sudah ada, dilewati.
Audio ID 9: Sudah ada, dilewati.
Audio ID 10: Sudah ada, dilewati.
Audio ID 11: Sudah ada, dilewati.
Audio ID 12: Sudah ada, dilewati.
Audio ID 13: Sudah ada, dilewati.
Audio ID 14: Sudah ada, dilewati.
Audio ID 15: Sudah ada, dilewati.
Audio ID 16: Sudah ada, dilewati.
Audio ID 17: Sudah ada, dilewati.
Audio ID 18: Sudah ada, dilewati.
Audio ID 19: Sudah ada, dilewati.
Audio ID 20: Sudah ada, dilewati.
Audio ID 21: Sudah ada, dilewati.
Audio ID 22: Sudah ada, dilewati.
Audio ID 23: Sudah ada, dilewati.
Audio ID 24: Sudah ada, dilewati.
Audio ID 25: Sudah ada, dilewati.
Audio ID 26: Sudah ada, dilewati.
Audio ID 27: Sudah ada, dilewati.
Audio ID 28: Sudah ada, dilewat

KeyboardInterrupt: 

In [16]:
from datasets import Dataset, Audio
import numpy as np

if os.path.exists(audio_test_path) and len(os.listdir(audio_test_path)) > 0:
    print(f"{len(os.listdir(audio_test_path))} audio test ready.")

    test_audio_files = os.listdir(audio_test_path)
    test_ids = [int(f.replace('audio_', '').replace('.wav', '')) for f in test_audio_files]
    df_test_audio = pd.DataFrame({
        'id': test_ids,
        'audio_path': [os.path.join(audio_test_path, f) for f in test_audio_files]
    })

    test_dataset = Dataset.from_pandas(df_test_audio)
    test_dataset = test_dataset.cast_column("audio_path", Audio(sampling_rate=16000))

    # no-labeling
    def preprocess_test_function(examples):
        audio_arrays = [x["array"] for x in examples["audio_path"]]
        inputs = feature_extractor(
            audio_arrays,
            sampling_rate=feature_extractor.sampling_rate,
            max_length=16000 * 5,
            truncation=True,
            padding=True
        )
        return inputs

    encoded_test_dataset = test_dataset.map(preprocess_test_function, remove_columns=["audio_path"], batched=True)

    print("\Successfully")

else:
    print("Failed")

Ditemukan 80 file audio di folder tes.

Memproses audio tes...


Map:   0%|          | 0/80 [00:00<?, ? examples/s]


✅ Data tes siap untuk diprediksi!


In [18]:
if 'encoded_test_dataset' in locals():
    test_predictions = trainer.predict(encoded_test_dataset)
    predicted_label_ids = np.argmax(test_predictions.predictions, axis=1)
    df_test_audio['predicted_id'] = predicted_label_ids
    display(df_test_audio.head())
else:
    print("ERROR: Dataset 'encoded_test_dataset' not found.")

Memulai prediksi pada 80 audio tes...



✅ Prediksi selesai! Berikut 5 hasil pertama:


,id,audio_path,predicted_id
0,1,/content/drive/My Drive/BDC_DATASET/audio/test...,0
1,2,/content/drive/My Drive/BDC_DATASET/audio/test...,0
2,3,/content/drive/My Drive/BDC_DATASET/audio/test...,0
3,4,/content/drive/My Drive/BDC_DATASET/audio/test...,0
4,5,/content/drive/My Drive/BDC_DATASET/audio/test...,0


In [21]:
import pandas as pd

if 'df_test_audio' in locals():
    official_label_map = {
        'Proud': 0, 'Trust': 1, 'Joy': 2, 'Surprise': 3,
        'Neutral': 4, 'Sadness': 5, 'Fear': 6, 'Anger': 7
    }

    df_test_audio['predicted_emotion'] = df_test_audio['predicted_id'].apply(lambda x: id2label[x])
    df_test_audio['predicted'] = df_test_audio['predicted_emotion'].apply(lambda x: official_label_map[x])

    submission_template_path = '/submission.csv'
    df_submission = pd.read_csv(submission_template_path)
    df_submission.columns = df_submission.columns.str.strip()

    df_submission = df_submission.drop(columns=['predicted'])
    final_submission_df = pd.merge(df_submission, df_test_audio[['id', 'predicted']], on='id', how='left')

    #most_frequent_class_id = 3
    #final_submission_df['predicted'] = final_submission_df['predicted'].fillna(most_frequent_class_id)

    final_submission_df['predicted'] = final_submission_df['predicted'].astype(int)

    submission_filename = 'submission_v1.csv'
    final_submission_df.to_csv(submission_filename, index=False)

    display(final_submission_df.head())

else:
    print("❌ ERROR: Dataframe 'df_test_audio' not found.")

Memulai pembuatan file submisi...

🎉 SELAMAT! File submisi Anda telah dibuat: 'submission_TimHebat_v1.csv'
File ini siap untuk diunggah ke platform lomba.

Berikut adalah 10 baris pertama dari file submisi Anda:


,id,predicted
0,1,3
1,2,3
2,3,3
3,4,3
4,5,3
5,6,3
6,7,3
7,8,3
8,9,3
9,10,3
